# 데이터 다운로드

In [1]:
!wget https://raw.githubusercontent.com/aifactory-team/AFCompetition/master/2144/baseline_keras_model.h5
!wget https://raw.githubusercontent.com/aifactory-team/AFCompetition/master/2144/baseline_keras_model_labels.txt
!wget https://raw.githubusercontent.com/aifactory-team/AFCompetition/master/2144/x_test.zip

--2022-09-28 19:27:11--  https://raw.githubusercontent.com/aifactory-team/AFCompetition/master/2144/baseline_keras_model.h5
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2453432 (2.3M) [application/octet-stream]
Saving to: ‘baseline_keras_model.h5’

baseline_keras_mode 100%[===================>]   2.34M  --.-KB/s    in 0.06s   

2022-09-28 19:27:11 (36.2 MB/s) - ‘baseline_keras_model.h5’ saved [2453432/2453432]

--2022-09-28 19:27:11--  https://raw.githubusercontent.com/aifactory-team/AFCompetition/master/2144/baseline_keras_model_labels.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... conn

In [2]:
!unzip  -o x_test.zip

Archive:  x_test.zip
   creating: x_test/
  inflating: x_test/92309.jpg        
  inflating: __MACOSX/x_test/._92309.jpg  
  inflating: x_test/.DS_Store        
  inflating: __MACOSX/x_test/._.DS_Store  
  inflating: x_test/83823.jpg        
  inflating: __MACOSX/x_test/._83823.jpg  
  inflating: x_test/29349.jpg        
  inflating: __MACOSX/x_test/._29349.jpg  
  inflating: x_test/12122.jpg        
  inflating: __MACOSX/x_test/._12122.jpg  
  inflating: x_test/23992.jpg        
  inflating: __MACOSX/x_test/._23992.jpg  
  inflating: x_test/32321.jpg        
  inflating: __MACOSX/x_test/._32321.jpg  
  inflating: x_test/59394.jpg        
  inflating: __MACOSX/x_test/._59394.jpg  
  inflating: x_test/12342.jpg        
  inflating: __MACOSX/x_test/._12342.jpg  
  inflating: x_test/88893.jpg        
  inflating: __MACOSX/x_test/._88893.jpg  
  inflating: x_test/88921.jpg        
  inflating: __MACOSX/x_test/._88921.jpg  
  inflating: x_test/77923.jpg        
  inflating: __MACOSX/x_test/

# 라벨 정보 불러오기

In [3]:
label_dict = {}

f = open('baseline_keras_model_labels.txt', 'r')

lines = f.readlines()
for line in lines:
    index, label_name = line.split(' ')
    label_dict[int(index)] = label_name.strip()
f.close()

print(label_dict)

{0: 'standing', 1: 'lyingonbelly', 2: 'lyingonside', 3: 'sitting'}


# 학습된 모델로 추론하기

In [5]:
from keras.models import load_model
from PIL import Image, ImageOps
import numpy as np
import os

# Load the model
model = load_model('baseline_keras_model.h5')

# Create the array of the right shape to feed into the keras model
# The 'length' or number of images you can put into the array is
# determined by the first position in the shape tuple, in this case 1.
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
# Replace this with the path to your image

folder_path = 'x_test'

y_pred = []

for fn in os.listdir(folder_path):
  if fn.startswith("."):
    continue

  fpath = os.path.join(folder_path, fn)

  image = Image.open(fpath)
  #resize the image to a 224x224 with the same strategy as in TM2:
  #resizing the image to be at least 224x224 and then cropping from the center
  size = (224, 224)
  image = ImageOps.fit(image, size, Image.ANTIALIAS)

  #turn the image into a numpy array
  image_array = np.asarray(image)
  # Normalize the image
  normalized_image_array = (image_array.astype(np.float32) / 127.0) - 1
  # Load the image into the array
  data[0] = normalized_image_array

  # run the inference
  prediction = model.predict(data)
  class_index = np.argmax(prediction)
  
  print(fn, label_dict[class_index])

  y_pred.append([fn, label_dict[class_index]])

23992.jpg standing
23482.jpg sitting
12122.jpg standing
29384.jpg sitting
88923.jpg lyingonbelly
88921.jpg lyingonbelly
92921.jpg lyingonside
92309.jpg standing
59394.jpg lyingonside
83823.jpg lyingonside
32321.jpg standing
77923.jpg lyingonbelly
12342.jpg lyingonbelly
43413.jpg standing
88893.jpg sitting
29349.jpg sitting


# 결과파일 생성 및 다운로드

In [6]:
f = open('y_pred.csv','wt')
for i in y_pred:
    f.write(i[0] + ', ' + i[1] + '\n')
f.close()

In [7]:
from google.colab import files
dfn = 'y_pred.csv'
files.download(dfn)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>